# Notebook 3: Data Aggregation
---

In [1]:
import datetime as dt
import numpy as np
import pandas as pd
from scipy.stats import zscore
import warnings
warnings.filterwarnings('ignore')

## Load the Citi Bike, Weather, Holiday, and NYC Bike Counts Data
---

In [2]:
# Load the data
tripdata = pd.read_parquet('../clean_data/hourly_tripdata.parquet')
weather = pd.read_parquet('../clean_data/clean_weather.parquet')
holiday = pd.read_parquet('../clean_data/holidays.parquet')
bike_counts = pd.read_parquet('../clean_data/nyc_bike_counts.parquet')

In [3]:
# Rename all indexes to 'date'
for df in [tripdata, weather, holiday, bike_counts]:
    df.index.name = 'date'

## Merge the Citi Bike, Weather, Holiday, and NYC Bike Counts Data
---

In [4]:
# Merge the DataFrames
df = pd.merge(tripdata, weather, on='date').merge(holiday, on='date').merge(bike_counts, on='date')

In [5]:
# Check for missing values
df.isna().sum()

nyc_trips_casual          0
nyc_trips_member          0
brooklyn_start_all        0
manhattan_start_all       0
queens_start_all          0
brooklyn_end_all          0
manhattan_end_all         0
queens_end_all            0
nyc_trips_all             0
brooklyn_start_casual     0
brooklyn_end_casual       0
brooklyn_start_member     0
brooklyn_end_member       0
manhattan_start_casual    0
manhattan_end_casual      0
manhattan_start_member    0
manhattan_end_member      0
queens_start_casual       0
queens_end_casual         0
queens_start_member       0
queens_end_member         0
temp                      0
humidity                  0
precip                    0
windspeed                 0
visibility                0
holidayName               0
isPaidTimeOff             0
isHoliday                 0
nyc_bike_counts           0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43824 entries, 2018-06-01 00:00:00 to 2023-05-31 23:00:00
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nyc_trips_casual        43824 non-null  float64
 1   nyc_trips_member        43824 non-null  float64
 2   brooklyn_start_all      43824 non-null  float64
 3   manhattan_start_all     43824 non-null  float64
 4   queens_start_all        43824 non-null  float64
 5   brooklyn_end_all        43824 non-null  float64
 6   manhattan_end_all       43824 non-null  float64
 7   queens_end_all          43824 non-null  float64
 8   nyc_trips_all           43824 non-null  int64  
 9   brooklyn_start_casual   43824 non-null  float64
 10  brooklyn_end_casual     43824 non-null  float64
 11  brooklyn_start_member   43824 non-null  float64
 12  brooklyn_end_member     43824 non-null  float64
 13  manhattan_start_casual  43824 non-null  float64
 14  man

In [7]:
df1 = df.drop(['holidayName', 'isPaidTimeOff', 'isHoliday'], axis=1)

In [8]:
# Check for outliers
zscore(df1)[(zscore(df1) < -3) | (zscore(df1) > 3)].count()

nyc_trips_casual          1125
nyc_trips_member           497
brooklyn_start_all         679
manhattan_start_all        504
queens_start_all           638
brooklyn_end_all           751
manhattan_end_all          489
queens_end_all             609
nyc_trips_all              526
brooklyn_start_casual     1142
brooklyn_end_casual       1128
brooklyn_start_member      531
brooklyn_end_member        572
manhattan_start_casual    1094
manhattan_end_casual      1088
manhattan_start_member     508
manhattan_end_member       491
queens_start_casual       1142
queens_end_casual         1113
queens_start_member        634
queens_end_member          606
temp                         6
humidity                     0
precip                     113
windspeed                  420
visibility                1748
nyc_bike_counts            417
dtype: int64

In [9]:
def winsorize_feature(feature, lower_percentile, upper_percentile):
    """
    Apply winsorization to a feature array.

    Args:
        feature (numpy.ndarray): The array containing the feature values.
        lower_percentile (float): The lower percentile value for winsorization.
        upper_percentile (float): The upper percentile value for winsorization.

    Returns:
        numpy.ndarray: The winsorized feature array with replaced outliers.
    """
    # Calculate the winsorizing values
    lower_value = np.percentile(feature, lower_percentile)
    upper_value = np.percentile(feature, upper_percentile)

    # Replace outliers with winsorizing values
    winsorized_feature = np.where(feature < lower_value, lower_value, feature)
    winsorized_feature = np.where(winsorized_feature > upper_value, upper_value, winsorized_feature)

    return winsorized_feature

In [10]:
df1.describe()

,nyc_trips_casual,nyc_trips_member,brooklyn_start_all,manhattan_start_all,queens_start_all,brooklyn_end_all,manhattan_end_all,queens_end_all,nyc_trips_all,brooklyn_start_casual,...,queens_start_casual,queens_end_casual,queens_start_member,queens_end_member,temp,humidity,precip,windspeed,visibility,nyc_bike_counts
count,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,...,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000
mean,434.835068,1720.539704,528.833607,1505.481449,121.059716,531.243656,1503.826282,120.304833,2155.374772,129.787080,...,23.164453,22.893369,97.895263,97.411464,57.503142,63.960504,0.003585,6.634465,9.399744,1567.002989
std,553.760156,1578.042468,510.203412,1413.383124,123.468301,520.092461,1408.740263,119.643284,2029.766138,168.058829,...,31.547424,30.635991,97.618336,94.994902,16.878033,18.458432,0.063389,4.096415,1.604257,1295.631593
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,5.900000,10.380000,0.000000,0.000000,0.000000,4.000000
25%,58.000000,376.000000,113.000000,327.000000,25.000000,114.000000,322.000000,26.000000,467.000000,16.000000,...,3.000000,3.000000,20.000000,21.000000,43.900000,49.860000,0.000000,4.000000,9.900000,460.750000
50%,212.000000,1350.000000,382.000000,1137.000000,81.000000,378.000000,1134.000000,81.000000,1609.000000,62.000000,...,10.000000,10.000000,69.000000,69.000000,57.000000,63.580000,0.000000,5.900000,9.900000,1296.000000
75%,595.000000,2613.000000,803.000000,2266.000000,179.000000,793.250000,2268.000000,179.000000,3256.000000,178.000000,...,31.000000,31.000000,147.000000,146.000000,71.900000,79.290000,0.000000,8.900000,9.900000,2357.250000
max,4207.000000,9937.000000,2841.000000,8697.000000,795.000000,3084.000000,8494.000000,760.000000,12055.000000,1167.000000,...,378.000000,270.000000,668.000000,663.000000,98.000000,100.000000,8.785000,31.300000,9.900000,8306.000000


In [11]:
# Winsorize outliers
for col in df1.columns:
    df1[col] = winsorize_feature(df1[col], 5, 95)

In [12]:
df1.describe()

,nyc_trips_casual,nyc_trips_member,brooklyn_start_all,manhattan_start_all,queens_start_all,brooklyn_end_all,manhattan_end_all,queens_end_all,nyc_trips_all,brooklyn_start_casual,...,queens_start_casual,queens_end_casual,queens_start_member,queens_end_member,temp,humidity,precip,windspeed,visibility,nyc_bike_counts
count,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,...,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000
mean,409.497399,1666.021545,511.169610,1459.518346,116.503126,512.076396,1459.317931,116.170820,2088.872882,121.559191,...,21.703633,21.412879,94.140585,93.766384,57.603160,64.137870,0.000759,6.511151,9.515907,1528.840065
std,471.810811,1423.282893,460.270717,1281.619290,110.051484,464.773109,1282.388587,107.581429,1841.623657,141.169608,...,26.661535,25.723596,86.219772,84.132988,16.005159,17.785818,0.002525,3.747648,1.128270,1191.391793
min,8.000000,60.000000,17.000000,52.000000,3.000000,17.000000,52.000000,3.000000,74.000000,2.000000,...,0.000000,0.000000,3.000000,3.000000,31.700000,33.831500,0.000000,0.200000,5.400000,92.000000
25%,58.000000,376.000000,113.000000,327.000000,25.000000,114.000000,322.000000,26.000000,467.000000,16.000000,...,3.000000,3.000000,20.000000,21.000000,43.900000,49.860000,0.000000,4.000000,9.900000,460.750000
50%,212.000000,1350.000000,382.000000,1137.000000,81.000000,378.000000,1134.000000,81.000000,1609.000000,62.000000,...,10.000000,10.000000,69.000000,69.000000,57.000000,63.580000,0.000000,5.900000,9.900000,1296.000000
75%,595.000000,2613.000000,803.000000,2266.000000,179.000000,793.250000,2268.000000,179.000000,3256.000000,178.000000,...,31.000000,31.000000,147.000000,146.000000,71.900000,79.290000,0.000000,8.900000,9.900000,2357.250000
max,1668.000000,4776.850000,1578.000000,4349.000000,374.000000,1603.000000,4336.000000,367.000000,6250.850000,498.000000,...,95.000000,91.000000,294.000000,286.000000,83.700000,92.780000,0.010000,14.500000,9.900000,4085.000000


In [13]:
df[df1.columns] = df1

In [14]:
# Save the merged data
df.to_parquet('../clean_data/compiled_data.parquet')